<a href="https://colab.research.google.com/github/mpanahi/detect_plagarism_persianText/blob/master/plagiarism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
f=open('drive/MyDrive/stopwords.txt',"r")
stopwords=[]
for line in f.readlines():
  stopwords.append(line.replace("\n"," "))
f.close()
orig= input('original:').replace("\n"," ")
#f2=open('suspicious text',"r")
plag=input('plag:').replace("\n"," ")
#f2.close()

original:ایرنا نوشت:رئیس جمهورگفت : در همین غرب و انگلیس در ۲۸ هفته ۸۱ زن دچار آسیب و تعدادی فوت شدند، آیا روی این موضوع هم حساسیت هست. آیا یک اطلاعیه صادر شد؟ سال گذشته در همین آمریکا ۱۱۰۰ نفر به دست پلیس کشته شدند چرا کسی برای آن اطلاعیه صادر نمی‌کند. چرا کسی سوال نمی‌کند این ۱۱۰۰ نفر چطور به دست پلیس کشته شدند.
plag:فارس نوشت : ارتش در بیانیه‌ای هجمه به خادمان مردم در فرماندهی انتظامی که همواره مجاهدانه و مخلصانه در جهت امنیت و آسایش مردم تلاش می‌کنند را محکوم کرد.  ارتش جمهوری اسلامی ایران در بیانیه‌ای، هرگونه تخریب اموال عمومی و اختلال در امنیت مردم و کشور و هجمه به حافظان امنیت را محکوم کرد.


In [ ]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 4.1 MB/s 
     |████████████████████████████████| 1.4 MB 48.8 MB/s 
     |████████████████████████████████| 233 kB 60.5 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=af94415fa6712baaa1f291c33bb77a9bd042757b83792d05dde92b822c17061c
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=153757 sha256=adcbfb6290ebebddbdc862fd7f0c792a2385ba184f9eb970d0b8afdce1e31b2e
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


In [ ]:
from hazm import *

normalizer = Normalizer()

In [ ]:

tokens_o=word_tokenize(normalizer.normalize(orig))
tokens_p=word_tokenize(normalizer.normalize(plag))

#lowerCase
tokens_o = [token for token in tokens_o]
tokens_p = [token for token in tokens_p]

punctuations=['"','.','(',')',',','?',';',':',"''",'``']
filtered_tokens_o = [w for w in tokens_o if not w in stopwords and not w in punctuations]
filtered_tokens_p = [w for w in tokens_p if not w in stopwords and not w in punctuations]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
#Trigram Similiarity measures
trigrams_o=[]
for i in range(len(tokens_o)-2):
    t=(tokens_o[i],tokens_o[i+1],tokens_o[i+2])
    trigrams_o.append(t)

s=0
trigrams_p=[]
for i in range(len(tokens_p)-2):
    t=(tokens_p[i],tokens_p[i+1],tokens_p[i+2])
    trigrams_p.append(t)
    if t in trigrams_o:
        s+=1

#jaccord coefficient = (S(o)^S(p))/(S(o) U S(p))
#J=s/(len(trigrams_o)+len(trigrams_p)) 
#print(J)

#containment measure
C=s/len(trigrams_p)
print(C)

0.0


In [ ]:
#longest common subsequence
#dynamic programming algorithm for finding lcs
def lcs(l1,l2):
    s1=word_tokenize(l1)
    s2=word_tokenize(l2)
    # storing the dp values 
    dp = [[None]*(len(s1)+1) for i in range(len(s2)+1)] 
  
    for i in range(len(s2)+1): 
        for j in range(len(s1)+1): 
            if i == 0 or j == 0: 
                dp[i][j] = 0
            elif s2[i-1] == s1[j-1]: 
                dp[i][j] = dp[i-1][j-1]+1
            else: 
                dp[i][j] = max(dp[i-1][j] , dp[i][j-1]) 
    return dp[len(s2)][len(s1)] 

In [ ]:
sent_o=sent_tokenize(orig)
sent_p=sent_tokenize(plag)

#maximum length of LCS for a sentence in suspicious text
max_lcs=0
sum_lcs=0

for i in sent_p:
    for j in sent_o:
        l=lcs(i,j)
        max_lcs=max(max_lcs,l)
    sum_lcs+=max_lcs
    max_lcs=0

score=sum_lcs/len(tokens_p)
print(score)

0.21052631578947367
